European Central Bank (ECB) API with Python
=====

## Data on Eurozone countries

-----

*December 14, 2019*<br>
*@bd_econ*

ECB API Documentation is [here](https://sdw-wsrest.ecb.europa.eu/help/)

This example requests the interest rate by country on 10-year bonds over the period since 2012.

You can read the output from the ECB API (as json data, so with some patience) in a web browser, to better understand what my code below is doing.

In [1]:
import requests
import pandas as pd

## Parameters/Settings

List of data sets is [here](https://sdw-wsrest.ecb.europa.eu/service/dataflow).

You can also find the `series_key` from searching the [Statistics Data Warehouse](https://sdw.ecb.europa.eu/home.do).

In [2]:
# List of countries
c_list = ['DE', 
          'FR', 
          'IT', 
          'ES', 
          'PT', 
          'GR', 
          'NL', 
          'BE', 
          'AT', 
          'IE']

param = [('dataflow', 'IRS'),
         ('freq', 'M'),
         ('countries', '+'.join(c_list)),
         ('series', 'L'),
         ('trans type', 'L40'),
         ('maturity cat', 'CI'),
         ('counterpart', '0000'),
         ('currency', 'EUR'),
         ('business coverage', 'N'),
         ('interest rate type', 'Z'),
         ('start', '?startPeriod=2012-01-01')]

param_joined = '.'.join(value for key, value in param[1:-1])

series_key = '{}/{}{}'.format(param[0][1],
                              param_joined,
                              param[-1][1])

In [3]:
print(series_key)

IRS/M.DE+FR+IT+ES+PT+GR+NL+BE+AT+IE.L.L40.CI.0000.EUR.N.Z?startPeriod=2012-01-01


## Request data

In [4]:
url = 'https://sdw-wsrest.ecb.europa.eu/service/data/'
# headers used as content negotiation to return data in json format
headers = {'Accept':'application/json'}
r = requests.get('{}{}'.format(url, series_key), headers=headers).json()

## Pandas DataFrame

In [5]:
date_list = r['structure']['dimensions']['observation'][0]['values']
dates = [date['start'][:10] for date in date_list]
    
areas = [v['name'] for v in r['structure']['dimensions']['series'][1]['values']]

df = pd.DataFrame()
for i, area in enumerate(areas):
    s_key = '0:{}:0:0:0:0:0:0:0'.format(i)
    s_list = r['dataSets'][0]['series'][s_key]['observations']
    df[area] = pd.Series([s_list[val][0] for val in sorted(s_list, key=int)])
df.index = dates
df.tail()

,Austria,Belgium,Germany,Spain,France,Greece,Ireland,Italy,Netherlands,Portugal
2020-02-01,-0.2574,-0.13,-0.470,0.272,-0.18,1.07,-0.1310,0.955,-0.335,0.247
2020-03-01,-0.0938,-0.02,-0.540,0.523,-0.06,1.97,0.0700,1.551,-0.326,0.706
2020-04-01,0.0458,0.14,-0.450,0.824,0.06,2.05,0.2012,1.799,-0.218,0.968
2020-05-01,-0.0974,0.04,-0.516,0.742,-0.03,1.93,0.1072,1.762,-0.312,0.808
2020-06-01,-0.1309,-0.04,-0.434,0.511,-0.04,1.32,0.0835,1.455,-0.293,0.527


### Example 2: Population

In [6]:
series_key2 = 'ENA/A.N.DE+FR+IT+ES+PT+GR+NL+BE+AT+IE.W0.S1.S1._Z.POP._Z._Z._Z.PS._Z.N'

In [7]:
url = 'https://sdw-wsrest.ecb.europa.eu/service/data/'
# headers used as content negotiation to return data in json format
headers = {'Accept':'application/json'}
r = requests.get(f'{url}{series_key2}', headers=headers).json()

In [8]:
date_list = r['structure']['dimensions']['observation'][0]['values']
dates = {i: v['id'] for i, v in enumerate(date_list)}    
areas = [v['name'] for v in r['structure']['dimensions']['series'][2]['values']]

df = pd.DataFrame()
for i, area in enumerate(areas):
    s_key = f'0:0:{i}:0:0:0:0:0:0:0:0:0:0:0'
    s_list = r['dataSets'][0]['series'][s_key]['observations']
    df[area] = pd.Series({dates[int(i)]: v[0] for i, v in s_list.items()})
df.tail()

,Austria,Belgium,Germany,Spain,France,Greece,Ireland,Italy,Netherlands,Portugal
2015,8629.519,11274,81687,46410.149,66581,10820.964,4695.769,60730.6,16940,10358.1
2016,8739.806,11331,82349,46449.874,66831,10775.989,4748.994,60627.5,17030,10325.5
2017,8795.073,11375,82657,46532.869,67063,10754.701,4802.274,60536.7,17131,10300.3
2018,8837.707,11427,82906,46728.962,67265,10732.894,4860.650,60458.7,17232,10283.8
2019,8875.607,11484,83093,47100.396,67456,10711.722,4927.173,60375.2,17345,10286.3


### Unemployment Rate

In [9]:
series_key3 = 'AME/A.DEU+FRA+ITA+ESP+PRT+GRC+NLD+BEL+AUT+IRL.1.0.0.0.ZUTN'
url = 'https://sdw-wsrest.ecb.europa.eu/service/data/'
# headers used as content negotiation to return data in json format
headers = {'Accept':'application/json'}
r = requests.get(f'{url}{series_key3}', headers=headers).json()

In [10]:
date_list = r['structure']['dimensions']['observation'][0]['values']
dates = {i: v['id'] for i, v in enumerate(date_list)}    
areas = [v['name'] for v in r['structure']['dimensions']['series'][1]['values']]

df = pd.DataFrame()
for i, area in enumerate(areas):
    s_key = f'0:{i}:0:0:0:0:0'
    s_list = r['dataSets'][0]['series'][s_key]['observations']
    df[area] = pd.Series({dates[int(i)]: v[0] for i, v in s_list.items()})
df.tail()

,Austria,Belgium,FR. Germany,Spain,France,Greece,Ireland,Italy,Netherlands,Portugal
2016,6.0,7.8,4.1,19.6,10.1,23.6,8.4,11.7,6.0,11.2
2017,5.5,7.1,3.8,17.2,9.4,21.5,6.7,11.2,4.9,9.0
2018,4.8,6.4,3.5,15.6,9.0,19.6,5.6,10.7,3.9,7.1
2019,4.6,6.1,3.2,14.4,8.8,18.2,5.1,10.4,3.6,6.3
2020,4.4,5.9,3.0,13.3,8.4,16.9,4.9,10.0,3.6,5.9
